In [52]:
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder
from allennlp.modules.text_field_embedders.basic_text_field_embedder import BasicTextFieldEmbedder
from allennlp.data.token_indexers.wordpiece_indexer import PretrainedBertIndexer
from allennlp.modules.token_embedders.bert_token_embedder import BertEmbedder
    from pytorch_pretrained_bert.modeling import BertConfig, BertModel


In [23]:
from allennlp.data.fields import TextField
from allennlp.data.tokenizers import Token

from allennlp.data.vocabulary import Vocabulary

In [4]:
from allennlp.data.tokenizers.word_splitter import BertBasicWordSplitter

In [5]:
bert_name = 'bert-base-uncased'

In [6]:
bert = PretrainedBertEmbedder(bert_name, top_layer_only=True)

In [48]:
embedder = BasicTextFieldEmbedder(token_embedders={'bert': bert},
                                  embedder_to_indexer_map={"bert": ["bert", "bert-offsets"]})

In [66]:
conf = BertConfig(vocab_size_or_config_json_file=30522)
bert_model = BertModel(conf)

In [67]:
embedder = BertEmbedder(bert_model)

In [68]:
tokenizer = WordTokenizer(word_splitter=BertBasicWordSplitter())

#            2   3    4   3     5     6   8      9    2   14   12
sentence1 = "the quickest quick brown fox jumped over the lazy dog"
tokens1 = tokenizer.tokenize(sentence1)

#            2   3     5     6   8      9    2  15 10 11 14   1
sentence2 = "the quick brown fox jumped over the laziest lazy elmo"
tokens2 = tokenizer.tokenize(sentence2)

vocab = Vocabulary()

instance1 = Instance({"tokens": TextField(tokens1, {"bert": self.token_indexer})})
instance2 = Instance({"tokens": TextField(tokens2, {"bert": self.token_indexer})})

batch = Batch([instance1, instance2])
batch.index_instances(vocab)

padding_lengths = batch.get_padding_lengths()
tensor_dict = batch.as_tensor_dict(padding_lengths)
tokens = tensor_dict["tokens"]

# 16 = [CLS], 17 = [SEP]
assert tokens["bert"].tolist() == [[16, 2, 3, 4, 3, 5, 6, 8, 9, 2, 14, 12, 17, 0],
                                   [16, 2, 3, 5, 6, 8, 9, 2, 15, 10, 11, 14, 1, 17]]

assert tokens["bert-offsets"].tolist() == [[1, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                                           [1, 2, 3, 4, 5, 6, 7, 10, 11, 12]]

# No offsets, should get 14 vectors back ([CLS] + 12 token wordpieces + [SEP])
bert_vectors = self.token_embedder(tokens["bert"])
assert list(bert_vectors.shape) == [2, 14, 12]

# Offsets, should get 10 vectors back.
bert_vectors = self.token_embedder(tokens["bert"], offsets=tokens["bert-offsets"])
assert list(bert_vectors.shape) == [2, 10, 12]

NameError: name 'WordTokenizer' is not defined

In [8]:
indexer = PretrainedBertIndexer(pretrained_model=bert_name, use_starting_offsets=True)

In [36]:
sentence = "The dog, Margharita ate his food, the food!"

In [9]:
splitter = BertBasicWordSplitter()

In [24]:
vocab = Vocabulary()

In [37]:
res = indexer.tokens_to_indices(splitter.split_words(sentence), vocabulary=vocab, index_name='bert')
res

{'bert': [101,
  1996,
  3899,
  1010,
  9388,
  5603,
  8486,
  2696,
  8823,
  2010,
  2833,
  1010,
  1996,
  2833,
  999,
  102],
 'bert-offsets': [1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14],
 'bert-type-ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [41]:
embedder(res)

ConfigurationError: "Your text field is generating more keys (['bert', 'bert-offsets', 'bert-type-ids', 'mask']) than you have token embedders (['bert']. If you are using a token embedder that requires multiple keys (for example, the OpenAI Transformer embedder or the BERT embedder) you need to add allow_unmatched_keys = True (and likely an embedder_to_indexer_map) to your BasicTextFieldEmbedder configuration. Otherwise, you should check that there is a 1:1 embedding between your token indexers and token embedders."

In [50]:
embedder({'bert': res['bert'], 'bert-offsets': res['bert-offsets']})

ConfigurationError: "Your text field is generating more keys (['bert', 'bert-offsets']) than you have token embedders (['bert']. If you are using a token embedder that requires multiple keys (for example, the OpenAI Transformer embedder or the BERT embedder) you need to add allow_unmatched_keys = True (and likely an embedder_to_indexer_map) to your BasicTextFieldEmbedder configuration. Otherwise, you should check that there is a 1:1 embedding between your token indexers and token embedders."